In [1]:
import json
import requests
import numpy as np
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import coint, adfuller
from statsmodels.api import OLS


In [2]:
def get_data(symbol):
    # Define the API URL and query parameters
    url = 'https://api.binance.com/api/v3/klines'
    params = {'symbol': symbol, 'interval': '1d', 'startTime': '1546300800000', 'endTime': '1609459200000', 'limit': '1000'}

    # Make a request to the API and load the response into a list
    response = requests.get(url, params=params)
    klines_list = json.loads(response.text)

    # Convert the list to a Pandas DataFrame
    columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']
    df = pd.DataFrame(klines_list, columns=columns)

    # Convert the timestamp column to a datetime object
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

    # Set the timestamp column as the DataFrame index
    df.set_index('timestamp', inplace=True)

    # Convert the numeric columns to float type
    numeric_columns = ['open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume']
    df[numeric_columns] = df[numeric_columns].astype(float)
    df = df.rename(columns={'close': symbol})
    
    # Return the DataFrame
    return df[symbol]

In [3]:
btc_df = get_data('BTCUSDT')
bnb_df = get_data('BNBUSDT')
eth_df = get_data('ETHUSDT')
ada_df = get_data('ADAUSDT')
ltc_df = get_data('LTCUSDT')


In [4]:
data = pd.concat([btc_df, eth_df, ada_df,ltc_df], axis=1)
data.columns = ['price_btc', 'price_eth', 'price_ada', 'price_ltc']
data = data.dropna()

In [5]:
data

,price_btc,price_eth,price_ada,price_ltc
timestamp,,,,
2019-01-01,3797.14,139.10,0.04198,31.48
2019-01-02,3858.56,152.01,0.04433,32.67
2019-01-03,3766.78,146.30,0.04205,31.39
2019-01-04,3792.01,151.97,0.04301,31.76
2019-01-05,3770.96,152.83,0.04403,34.33
...,...,...,...,...
2020-12-28,27079.41,730.41,0.17627,129.83
2020-12-29,27385.00,732.00,0.19067,128.80
2020-12-30,28875.54,752.17,0.18389,130.05


In [6]:
train_data = data.loc['2019-01-01':'2020-7-31']
test_data = data.loc['2020-08-01':]

In [7]:
def engle_granger_two_step(data, dependent_var, independent_vars):
    Y = data[dependent_var]
    X = data[independent_vars]
    X = sm.add_constant(X)
    
    # Step 1: Estimate the cointegrating relationship
    model = sm.OLS(Y, X).fit()
    residuals = model.resid
    
    # Step 2: Test for the presence of a unit root in the residuals
    adf_test = adfuller(residuals)
    
    return model, residuals, adf_test


In [8]:
dependent_var = 'price_btc'
independent_vars = ['price_eth', 'price_ada', 'price_ltc']

model, residuals, adf_test = engle_granger_two_step(train_data, dependent_var, independent_vars)
model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                    Results: Ordinary least squares
=======================================================================
Model:                OLS               Adj. R-squared:      0.710     
Dependent Variable:   price_btc         AIC:                 9889.7389 
Date:                 2023-05-06 15:02  BIC:                 9907.1772 
No. Observations:     578               Log-Likelihood:      -4940.9   
Df Model:             3                 F-statistic:         471.8     
Df Residuals:         574               Prob (F-statistic):  1.95e-154 
R-squared:            0.711             Scale:               1.5683e+06
-----------------------------------------------------------------------
             Coef.     Std.Err.    t     P>|t|     [0.025      0.975]  
-----------------------------------------------------------------------
const        848.4049  203.2389   4.1744 0.0000    449.2223   1247.5876
price_eth     54.9984    1.7431  31.5519 0.0000     51.5748     58.4221
price_ada -49734.0655 3191.5254 -15.5832 0.0000 -56002.5578 -43465.5731
price_ltc     -9.1675    2.6816  -3.4187 0.0007    -14.4345     -3.9006
-----------------------------------------------------------------------
Omnibus:                22.854         Durbin-Watson:            0.036 
Prob(Omnibus):          0.000          Jarque-Bera (JB):         10.794
Skew:                   0.082          Prob(JB):                 0.005 
Kurtosis:               2.351          Condition No.:            12639 
=======================================================================
* The condition number is large (1e+04). This might indicate
strong multicollinearity or other numerical problems.
"""

In [9]:
adf_stat = adf_test[0]
adf_critical_value = adf_test[4]['5%']

if adf_stat < adf_critical_value:
    print("The model is valid.")
else:
    print("The model is not valid.")


The model is valid.


In [10]:
# Calculate the spread for the training and test datasets
train_spread = train_data['price_btc'] + 51.78 * train_data['price_eth'] - 50972.52 * train_data['price_ada'] - 4.41 * train_data['price_ltc']
test_spread  = test_data['price_btc']  + 51.78 * test_data['price_eth']  - 50972.52 * test_data['price_ada']  - 4.41 * test_data['price_ltc']

In [11]:
lookback = 5
entry_threshold = 1
# Create a new DataFrame for the test spread
test_df = pd.DataFrame(test_spread, columns=['spread_portfolio'])

# Calculate the moving average and standard deviation for the test spread using the same lookback window
test_df['moving_avg'] = test_df['spread_portfolio'].rolling(window=lookback).mean()
test_df['std_dev'] = test_df['spread_portfolio'].rolling(window=lookback).std()

# Create buy and sell signals based on mean reversion for the test spread
test_df['signal'] = np.where(test_df['spread_portfolio'] < test_df['moving_avg'] - entry_threshold * test_df['std_dev'], 'BUY',
                             np.where(test_df['spread_portfolio'] > test_df['moving_avg'] + entry_threshold * test_df['std_dev'], 'SELL', 'HOLD'))

# Print the test DataFrame with the buy and sell signals
print(test_df)

            spread_portfolio    moving_avg      std_dev signal
timestamp                                                     
2020-08-01      24223.013368           NaN          NaN   HOLD
2020-08-02      23287.327350           NaN          NaN   HOLD
2020-08-03      23947.359404           NaN          NaN   HOLD
2020-08-04      23805.004958           NaN          NaN   HOLD
2020-08-05      24982.670380  24049.075092   622.902605   SELL
...                      ...           ...          ...    ...
2020-12-28      55342.563400  51044.375870  3352.842586   SELL
2020-12-29      55001.021612  52615.841385  2874.822651   HOLD
2020-12-30      57876.045397  54476.666279  2601.034804   SELL
2020-12-31      57264.334723  55758.550093  1837.077532   HOLD
2021-01-01      57593.152873  56615.423601  1340.958087   HOLD

[154 rows x 4 columns]


In [12]:
# Initialize variables
test_df['position'] = None
test_df['pnl'] = 0.0
position = 0

# Iterate through the test_df DataFrame
for i, row in test_df.iterrows():
    # If there's a BUY signal and you're not in a position, enter the position
    if row['signal'] == 'BUY' and position == 0:
        position = 1
        entry_price = row['spread_portfolio']
        test_df.loc[i, 'position'] = position

    # If there's a SELL signal and you're in a position, exit the position
    elif row['signal'] == 'SELL' and position == 1:
        position = 0
        exit_price = row['spread_portfolio']
        pnl = exit_price - entry_price
        test_df.loc[i, 'pnl'] = pnl
        test_df.loc[i, 'position'] = position

    # If you're holding the position, update the position status
    elif position != 0:
        test_df.loc[i, 'position'] = position

# Calculate the cumulative return
test_df['cumulative_return'] = test_df['pnl'].cumsum()


,spread_portfolio,moving_avg,std_dev,signal,position,pnl,cumulative_return
timestamp,,,,,,,
2020-08-01,24223.013368,NaN,NaN,HOLD,None,0.0,0.000000
2020-08-02,23287.327350,NaN,NaN,HOLD,None,0.0,0.000000
2020-08-03,23947.359404,NaN,NaN,HOLD,None,0.0,0.000000
2020-08-04,23805.004958,NaN,NaN,HOLD,None,0.0,0.000000
2020-08-05,24982.670380,24049.075092,622.902605,SELL,None,0.0,0.000000
...,...,...,...,...,...,...,...
2020-12-28,55342.563400,51044.375870,3352.842586,SELL,None,0.0,17402.551472
2020-12-29,55001.021612,52615.841385,2874.822651,HOLD,None,0.0,17402.551472
2020-12-30,57876.045397,54476.666279,2601.034804,SELL,None,0.0,17402.551472


In [13]:
test_df.head(50)


,spread_portfolio,moving_avg,std_dev,signal,position,pnl,cumulative_return
timestamp,,,,,,,
2020-11-13,35281.492649,33913.745931,1095.288233,SELL,None,0.000000,5656.638302
2020-11-14,34347.006643,34244.969767,857.672886,HOLD,None,0.000000,5656.638302
2020-11-15,33763.228241,34405.144609,591.840982,BUY,1,0.000000,5656.638302
2020-11-16,34940.698948,34593.467978,580.022894,HOLD,1,0.000000,5656.638302
2020-11-17,36708.574036,35008.200103,1113.288873,SELL,0,2945.345795,8601.984097
2020-11-18,36791.756418,35310.252857,1379.116877,SELL,None,0.000000,8601.984097
2020-11-19,36446.607832,35730.173095,1331.319625,HOLD,None,0.000000,8601.984097
2020-11-20,38676.696634,36712.866774,1329.881795,SELL,None,0.000000,8601.984097
2020-11-21,40123.519783,37749.430941,1596.401511,SELL,None,0.000000,8601.984097
